In [1]:
import numpy as np
from scipy.signal import get_window
import math
import sys, os
sys.path.append('../software/models/')
import dftModel as DFT
import utilFunctions as UF
import stft
import sineModel as SM
import IPython.display as ipd

In [2]:
from scipy.io.wavfile import write, read
INT16_FAC = (2**15)-1
INT32_FAC = (2**31)-1
INT64_FAC = (2**63)-1
norm_fact = {'int16':INT16_FAC, 'int32':INT32_FAC, 'int64':INT64_FAC,'float32':1.0,'float64':1.0}
def wavread(filename):
    fs, x = read(filename)
    x = np.float32(x)/norm_fact[x.dtype.name]
    return fs, x

# STFT

### RTMA helpers 

In [3]:
from scipy.signal import check_COLA

In [4]:
#export
def gen_sinusoid(amp = 0.5,
                 freq = 440.0,
                 len_seconds = None,
                 num_samples = None,
                 sample_rate = 44100,
                 phi = 0.0):
    if num_samples is None and len_seconds is not None:
        num_samples = len_seconds * sample_rate
    if num_samples is None:
        num_samples = sample_rate

    if isinstance(freq, list):
        freq = np.linspace(freq[0], freq[1], num_samples)

    n = np.arange(num_samples)
    x = amp * np.cos(2.0 * np.pi * freq * n / sample_rate + phi)
    return x

In [5]:
#export
def get_cola_window(window_name: str, n_window: int, n_hop: int):
    even_window = n_window % 2 == 0
    window_by_hop = n_window // n_hop

    if (window_name == 'hamming'):
        window = get_window(window_name, n_window, fftbins=even_window)
        if not even_window:
            window[ 0] /= 2
            window[-1] /= 2
        window /= (window_by_hop * 0.54)
        
    elif (window_name == 'hann' or window_name == 'hanning'):
        window = get_window(window_name, n_window, fftbins=even_window)
    
    elif (window_name == 'blackman'):
        window = get_window(window_name, n_window, fftbins=even_window)
        window /= (window_by_hop * 0.42)
        
    n_overlap = n_window - n_hop
    assert check_COLA(window, nperseg=n_window, noverlap=n_overlap)
    
    return window

## Comparisons

In [6]:
import librosa as lr

Parameters:

In [7]:
fn = "/home/john/code/repos/rtma/nbs/data/E_octaves_both.wav"
(fs, x) = wavread(fn)
fs, x.shape, x.dtype

(48000, (271522,), dtype('float32'))

In [22]:
m = 1001
H = 512
N = 2048
w = get_window('hamming', m)
w_norm = w / w.sum()
w.shape, w.sum(), w.max(), w_norm.sum()

((1001,), 540.5400000000001, 0.9999977345260643, 0.9999999999999999)

### librosa

In [21]:
X = lr.stft(x, n_fft=N, hop_length=H, win_length=m, window=w, pad_mode='constant')
mx = abs(X)
mx = 20.0 * np.log10(mx)
mx.min(), mx.max()

(-172.58548, -15.702148)

In [24]:
X = lr.stft(x, n_fft=N, hop_length=H, win_length=m, window=w_norm, pad_mode='constant')
mx = abs(X)
mx = 20.0 * np.log10(mx)
mx.min(), mx.max()

(-172.58548, -15.702148)

In [23]:
X = lr.stft(x, n_fft=N, hop_length=H, win_length=m, window='hamming', pad_mode='constant')
mx = abs(X)
mx = 20.0 * np.log10(mx)
mx.min(), mx.max()

(-117.92894, 38.954407)

### sms-tools

In [15]:
mx, px = stft.stftAnal(x, w, N, H)
mx.min(), mx.max()

(-174.9114915763514, -15.697454732342324)

In [25]:
mx, px = stft.stftAnal(x, w_norm, N, H)  # no difference, stftAnal normalizes internally
mx.min(), mx.max()

(-174.91149157636949, -15.697454732342324)

### rtma

In [ ]:
w = get_window('hamming', 2048)
N = 4096
t = -80

In [ ]:
y = SM.sineModel(x, fs, w, N, t)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x)

In [ ]:
plt.plot(y)

In [ ]:
ipd.Audio(x, rate=fs)

In [ ]:
ipd.Audio(y, rate=fs)

In [ ]:
sr = 100
amp = 0.5

n_fft = 128

freq = [sr / n_fft * f for f in [3, 17]]

num_samples = 200
x = gen_sinusoid(amp, freq=freq, sample_rate=sr, num_samples=num_samples)

plot(x)

In [ ]:
#export
def gen_sinusoid(amp = 0.5,
                 freq = 440.0,
                 len_seconds = None,
                 num_samples = None,
                 sample_rate = 44100,
                 phi = 0.0):
    if num_samples is None and len_seconds is not None:
        num_samples = len_seconds * sample_rate
    if num_samples is None:
        num_samples = sample_rate

    if isinstance(freq, list):
        freq = np.linspace(freq[0], freq[1], num_samples)

    n = np.arange(num_samples)
    x = amp * np.cos(2.0 * np.pi * freq * n / sample_rate + phi)
    return x

In [ ]:
#export
def plot(x):
    plt.figure(figsize=(10, 2))
    plt.plot(x)

In [ ]:
stft?

In [ ]:
stft.stftAnal

In [ ]:
mx, px = stft.stftAnal(x, w, n_fft, h)
mx.max()

In [ ]:
m = 97
h = 48
w = get_cola_window('hamming', m, h)
plot(w)
wx = x[:m] * w
plot(wx)